# Pytorch autograd.grad 函数研究

本 notebook 将对 Pytorch 库中的 **autograd.grad** 函数行为进行详细探究。此文撰写时（2022.06.15） Pytorch 版本为 1.11.0

首先查看其[官方文档](https://pytorch.org/docs/stable/generated/torch.autograd.grad.html)，翻译如下：

## TORCH.AUTOGRAD.GRAD
```python
torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=False, is_grads_batched=False)
``` 
[SOURCE 源代码](https://pytorch.org/docs/stable/_modules/torch/autograd.html#grad)

> Computes and returns the sum of gradients of outputs with respect to the inputs.

计算并返回 输出相对于输入的 梯度的 求和（完全搞不清楚这在说什么）

> grad_outputs should be a sequence of length matching output containing the “vector” in vector-Jacobian product, usually the pre-computed gradients w.r.t. each of the outputs. If an output doesn’t require_grad, then the gradient can be None).

参数 “grad_outputs” 应该是一个序列，长度对应于输出，每个元素是 “向量雅可比积（vector-Jacobian product）” 中的 “向量”。此 “向量”序列 通常代表着先前计算的 关于每个输出的 梯度序列。如果一个输出没有 require_grad 标记，那么其梯度可以是 None。（同样也没看明白）

NOTE 注意
> If you run any forward ops, create grad_outputs, and/or call grad in a user-specified CUDA stream context, see Stream semantics of backward passes.
> 如果在用户指定的CUDA流上下文中运行任何正向运算、创建grad_outputs和/或调用grad，请参阅向后传递的流语义。（完全不知所云）

NOTE 注意
> only_inputs argument is deprecated and is ignored now (defaults to True). To accumulate gradient for other parts of the graph, please use torch.autograd.backward.
> 参数 “only_inputs” 已弃用，现在将被忽略(默认为True)。要累加计算图中其他部分的梯度，请使用 “torch.autograd.backward”。

Parameters 参数

> outputs (sequence of Tensor) – outputs of the differentiated function.

outputs (Tensor 的序列) – 可微函数的输出序列。

> inputs (sequence of Tensor) – Inputs w.r.t. which the gradient will be returned (and not accumulated into .grad).

inputs (Tensor 的序列) – 应该被计算梯度的输入序列，这些梯度将成为函数的返回值（这些梯度不会被累加到 .grad 中去）

> grad_outputs (sequence of Tensor) – The “vector” in the vector-Jacobian product. Usually gradients w.r.t. each output. None values can be specified for scalar Tensors or ones that don’t require grad. If a None value would be acceptable for all grad_tensors, then this argument is optional. Default: None.

grad_outputs (Tensor 的序列) – “向量雅可比积（vector-Jacobian product）” 中的 “向量”。通常代表着先前计算的 关于每个输出的 梯度序列。对于标量输出或者不需要计算梯度的输出，“向量”可以为 None。 若对于所有的输出其对应的“向量”都可以是 None，那么该参数可以直接忽略，不指定. 默认值：None。

> retain_graph (bool, optional) – If False, the graph used to compute the grad will be freed. Note that in nearly all cases setting this option to True is not needed and often can be worked around in a much more efficient way. Defaults to the value of create_graph.

retain_graph (布尔值, 可选) – 如果为 False，则用于计算梯度的计算图所占内存将被释放。请注意，在几乎所有情况下，都不需要将此选项设置为 True，此时可以使该函数效率更高（省内存）。默认为参数 create_graph 的值。

> create_graph (bool, optional) – If True, graph of the derivative will be constructed, allowing to compute higher order derivative products. Default: False.

create_graph (布尔值, 可选) – 如果为 True，则将同时构造此导数的计算图，从而允许计算更高阶导数。默认值：False。

> allow_unused (bool, optional) – If False, specifying inputs that were not used when computing outputs (and therefore their grad is always zero) is an error. Defaults to False.

allow_unused (布尔值, 可选) – 如果为 False，则在计算输出对应某个输入的梯度时，若该输入未在该输出的计算图中(因此它们的梯度始终为零)，就会报错。默认为 False。

> is_grads_batched (bool, optional) – If True, the first dimension of each tensor in grad_outputs will be interpreted as the batch dimension. Instead of computing a single vector-Jacobian product, we compute a batch of vector-Jacobian products for each “vector” in the batch. We use the vmap prototype feature as the backend to vectorize calls to the autograd engine so that this computation can be performed in a single call. This should lead to performance improvements when compared to manually looping and performing backward multiple times. Note that due to this feature being experimental, there may be performance cliffs. Please use torch._C._debug_only_display_vmap_fallback_warnings(True) to show any performance warnings and file an issue on github if warnings exist for your use case. Defaults to False.

is_grads_batched (bool, optional) – 如果为 True，则 grad_outputs 中每个张量的第一维将被解释为批次维。我们不是计算单个向量雅可比乘积，而是为批次中的每个“向量”计算一批向量雅可比乘积。我们使用 vmap 原型功能作为后端来向量化对 autograd 引擎的调用，以便可以在单个调用中执行此计算。与手动循环和多次向后执行相比，这应该会带来性能改进。请注意，由于此功能处于实验阶段，因此可能会出现性能骤降。如果您的用例存在警告，请使用torch._C._debug_only_display_vmap_fallback_warnings(True) 显示任何性能警告，并在github上提交问题。默认为 False。

看完此文档，感觉对函数用法以及每一个参数都做了解释，但是又含糊不清。看来还是需要进行进一步实验来确定其行为。


## 标量函数

### 对 outputs 与 inputs 参数的研究

In [72]:
import torch
from torch.autograd import grad

def f(x):
    return 2.0 * x + 1.0

x = torch.tensor(1.0)
x.requires_grad_()

grad(f(x),x) # (tensor(2.),)
grad(f(x),(x,x)) # (tensor(2.), tensor(2.))
grad((f(x),f(x)),x) # (tensor(4.),)
grad(f(x) + f(x),x) # (tensor(4.),)
grad((f(x),f(x)),(x,x)) # (tensor(4.), tensor(4.))
grad((f(x),x),(x,x)) # (tensor(3.), tensor(3.))
grad((x,f(x)),(x,x)) # (tensor(3.), tensor(3.))

(tensor(3.), tensor(3.))

观察以上输出，结合源代码，做出如下猜想：
1. 输入的 outputs 与 inputs 若不是元组，则首先会处理成一个单元素的元组
2. 输出一个元组，且长度与输入的 inputs 参数元组长度相等，每一个输出分量对应于 inputs 每一个分量
3. 输入的 outputs 有多个的话则梯度会累加，换句话说，程序自动把多个 outputs 相加了，再计算每个 inputs 的梯度

第 3. 点引出一个问题，它会累积梯度是因为重复计算吗，那如果预先计算好会怎样：

In [73]:
grad(f(x),x) # (tensor(2.),)
y = f(x)
# grad(y + y,x) # (tensor(4.),)
grad((y,y),x) # (tensor(4.),)

(tensor(4.),)

答案是依然会累加

In [29]:
import torch
from torch.autograd import grad

def f(x1,x2):
    return x1 * x2 + 3 * x1 + 5 * x2

x1 = torch.tensor(0.5)
x1.requires_grad_()
x2 = torch.tensor(1.5)
x2.requires_grad_()

print('grad(f(x1,x2),x1):',grad(f(x1,x2),x1)) # (tensor(4.5000),)
print('grad(f(x1,x2),x2):',grad(f(x1,x2),x2)) # (tensor(5.5000),)
print('grad(f(x1,x2),(x1,x2)):',grad(f(x1,x2),(x1,x2))) # (tensor(4.5000), tensor(5.5000))
print('grad((f(x1,x2),x1 + x2),(x1,x2)):',grad((f(x1,x2),x1 + x2),(x1,x2))) # (tensor(5.5000), tensor(6.5000))

grad(f(x1,x2),x1): (tensor(4.5000),)
grad(f(x1,x2),x2): (tensor(5.5000),)
grad(f(x1,x2),(x1,x2)): (tensor(4.5000), tensor(5.5000))
grad((f(x1,x2),x1 + x2),(x1,x2)): (tensor(5.5000), tensor(6.5000))


可以看出以上猜想依然成立

### 对 allow_unused 参数的研究

In [21]:
print('grad(2 * x1 + 0.0 * x2,(x1,x2)):',grad(2 * x1 + 0.0 * x2,(x1,x2))) # (tensor(2.), tensor(0.))
# print('grad(2 * x1,(x1,x2)):',grad(2 * x1,(x1,x2))) 报错 One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.
print('grad(2 * x1,(x1,x2),allow_unused=True):',grad(2 * x1,(x1,x2),allow_unused=True)) # (tensor(2.), None)

grad(2 * x1 + 0.0 * x2,(x1,x2)): (tensor(2.), tensor(0.))
grad(2 * x1,(x1,x2),allow_unused=True): (tensor(2.), None)


可以看出，若某一输出不在计算图中，则需要令 allow_unused=True 才不会报错，且对应的梯度输出为 None

### 对 grad_outputs 研究

In [33]:
import torch
from torch.autograd import grad

def f(x):
    return 2.0 * x + 1.0

x = torch.tensor(1.0)
x.requires_grad_()

print(grad(f(x),x,grad_outputs=(torch.tensor(0.3),))) # (tensor(0.6000),)
print(grad(f(x),(x,x),grad_outputs=(torch.tensor(0.3),))) # (tensor(0.6000), tensor(0.6000))
print(grad(f(x),(x,x),grad_outputs=(torch.tensor(0.3),torch.tensor(0.6)))) # (tensor(0.6000), tensor(0.6000))
print(grad(f(x),(x,x),grad_outputs=(torch.tensor(0.6),torch.tensor(0.3)))) # (tensor(1.2000), tensor(1.2000))
print(grad((f(x),x),(x,x),grad_outputs=(torch.tensor(0.6),torch.tensor(0.3)))) # (tensor(1.5000), tensor(1.5000))
print(grad((f(x),x),(x,x),grad_outputs=(torch.tensor(0.3),torch.tensor(0.6)))) # (tensor(1.2000), tensor(1.2000))
print(grad((x,f(x)),(x,x),grad_outputs=(torch.tensor(0.6),torch.tensor(0.3)))) # (tensor(1.2000), tensor(1.2000))


(tensor(0.6000),)
(tensor(0.6000), tensor(0.6000))
(tensor(0.6000), tensor(0.6000))
(tensor(1.2000), tensor(1.2000))
(tensor(1.5000), tensor(1.5000))
(tensor(1.2000), tensor(1.2000))
(tensor(1.2000), tensor(1.2000))


做出猜测，grad_outputs 对应于 outputs 的每一个元素，在计算梯度时会将对应梯度相乘

## 向量函数

### 对 outputs 与 inputs 参数的研究

In [2]:
import torch
from torch.autograd import grad

W = torch.tensor([[1,2],[3,4],[5,6]],dtype=torch.float) # 3 x 2
b = torch.tensor([[7,8]],dtype=torch.float) # 1 x 2

x =torch.tensor([[1.1,1.2,1.3]]) # 1 x 3
x.requires_grad_()

# grad(x @ W + b,x) # grad can be implicitly created only for scalar outputs
grad((x @ W + b).sum(),x) # (tensor([[ 3.,  7., 11.]]),) 输出和 inputs 形状一致的 梯度向量
grad((x @ W + b).sum(),(x,x)) # (tensor([[ 3.,  7., 11.]]), tensor([[ 3.,  7., 11.]]))
grad(((x @ W + b).sum(),x.sum()),x) # (tensor([[ 4.,  8., 12.]]),)
grad((x @ W + b).sum() + x.sum(),x) # (tensor([[ 4.,  8., 12.]]),)

# y = (x @ W + b).sum()
# grad(y,x) # (tensor([[ 3.,  7., 11.]]),)
# grad((y,y),x) 3 (tensor([[ 6., 14., 22.]]),)
y = (x @ W + b)
# grad([y[:,0],y[:,1]],x) #(tensor([[ 3.,  7., 11.]]), tensor([[ 3.,  7., 11.]]))
# grad([y[0,0],y[0,1]],x) # (tensor([[ 3.,  7., 11.]]),)
# grad([y[:,0]],x) # (tensor([[1., 3., 5.]]),)
grad(y,x,grad_outputs=(torch.tensor([[1,1]]),)) # (tensor([[ 3.,  7., 11.]]),)


(tensor([[ 3.,  7., 11.]]),)

In [17]:
import torch
from torch.autograd import grad



def f1(x:torch.Tensor):
    W = torch.tensor([[1,2],[3,4],[5,6]],dtype=torch.float) # 3 x 2
    b = torch.tensor([[7,8]],dtype=torch.float) # 1 x 2
    y = x @ W + b
    return y

def f2(x:torch.Tensor):
    W = torch.tensor([[1,2],[3,4],[5,6],[7,8]],dtype=torch.float) # 3 x 2
    b = torch.tensor([[9,10]],dtype=torch.float) # 1 x 2
    y = x @ W + b
    return y

x1 = torch.rand(size = ())
x1 = torch.tensor([1.1,1.2,1.3])
x1.requires_grad_()
x2 = torch.tensor([2.1,2.2,2.3,2.4])
x2.requires_grad_()

y1 = f1(x1)
y2 = f2(x2)

grad((y1,y2),(x2),grad_outputs=(torch.tensor([[0.5,0.5]]),torch.tensor([[1,1]])))

RuntimeError: got 2 tensors and 1 gradients

综上，基本上摸清了 Pytorch 中 autograd.grad 函数的行为，总结如下：

函数调用样例为：

```python
import torch

def f1(x:torch.Tensor):
    ...
    return y:torch.Tensor

def f2(x:torch.Tensor):
    ...
    return y:torch.Tensor

def f3(x:torch.Tensor):
    ...
    return y:torch.Tensor

x1_shape = (1,m1)
x2_shape = (1,m2)
x3_shape = (1,m3)

y1_shape = (1,n1)
y2_shape = (1,n2)
y3_shape = (1,n3)

x1 = torch.rand(size = x1_shape)
x2 = torch.rand(size = x1_shape)
x3 = torch.rand(size = x1_shape)
for x in (x1,x2,x3):
    x.requires_grad_()

y1 = f1(x1)
y2 = f2(x2)
y3 = f3(x3)

v1 = torch.rand(size = y1_shape)
v2 = torch.rand(size = y2_shape)
v3 = torch.rand(size = y3_shape)

results = torch.autograd.grad(
    outputs = (y1,y2,y3), 
    inputs = (x1,x2), 
    grad_outputs=(v1,v2,v3), 
    retain_graph=None, 
    create_graph=False, 
    only_inputs=True, 
    allow_unused=False, 
    is_grads_batched=False
)
```

参考以上样例，对于如下的调用：
```python
(g1,g2,...,gm) = torch.autograd.grad(
    outputs = (y1,y2,...,yn), 
    inputs = (x1,x2,...,xm), 
    grad_outputs=(v1,v2,...vn), 
    retain_graph=None, 
    create_graph=False, 
    only_inputs=True, 
    allow_unused=False, 
    is_grads_batched=False
)
```
要求：
1. 任意 xi， 都需要执行 xi.requires_grad_()
2. 任意 xi，需存在 yj，使得 xi 参与了 yj 的计算，否则需要设置 allow_unused=True 才不会报错，此时输出的对应 gi 为 None。
3. 任意 yi，对应一个 vi，两者形状需相同。当 yi 为标量时，vi 可以为 None，此时 vi 自动取 1.0；当所有的 yi 均为标量时，grad_outputs 可以取 None（这也是其默认值），此时 grad_outputs 自动取 (1.0,1.0,...1.0) 【n个】。
4. 函数执行一遍后若再次对 yi 求偏导会报错，因为执行一遍后会删除计算图，除非设置 retain_graph=True。
5. 若之后想求 gi 对 xj 的偏导（也就是高阶导）会报错，因为 gi 没有 连接 xj 的计算图，除非设置 create_graph=True（此时 retain_graph=create_graph=True）
6. only_inputs 参数没有效果，已被弃用。
7. is_grads_batched 暂未测试。

程序行为：
函数会输出 m 个向量的元组 (g1,g2,...,gm)，其中 gi 代表 :
$$
    \vec{g}_i = \left(\frac{\partial \sum_{j=1}^{n} \vec{v}_j \cdot \vec{y}_j}{\partial \vec{x}_{i_1}}, \cdots, \frac{\partial \sum_{j=1}^{n} \vec{v}_j \cdot \vec{y}_j}{\partial \vec{x}_{i_s}}\right) 
$$
文字描述一下，也就是说，首先程序会将 vi 与 yi 两两做内积然后相加，得到合并的标量输出 y，然后对每一个 xj 的每一个分量 xjk，计算 y 对 xjk 的偏导数，最后把 xjk 组装成 gj；m 个 xj 对应 m 个 gj，合并成一个元组后返回。

个人总结：
基于以上说明，我自己在使用 torch.autograd.grad() 函数时，首先 outputs 永远只输入一个标量值，同时永远忽略 grad_outputs 参数（也就是让其默认为 1.0）。因为如果有需要自己完全可以在外面做好向量内积相加等操作，为何交给这个行为不写明的函数来做呢。

其次在写通用的求导工具函数时，allow_unused 永远设为 True，然后自行将 None 处理成 0，因为如果某个 xi 没有被使用，其导数数学意义就是 0，让他报错干嘛。

完毕。



In [17]:
import torch
from torch.autograd import grad

def f1(x:torch.Tensor):
    W = torch.tensor([[1,2],[3,4],[5,6]],dtype=torch.float) # 3 x 2
    b = torch.tensor([[7,8]],dtype=torch.float) # 1 x 2
    y = x @ W + b
    return y

def f2(x:torch.Tensor):
    W = torch.tensor([[1,2],[3,4],[5,6],[7,8]],dtype=torch.float) # 3 x 2
    b = torch.tensor([[9,10]],dtype=torch.float) # 1 x 2
    y = x @ W + b
    return y



x1 = torch.rand(size = ())
x1 = torch.tensor([1.1,1.2,1.3])
x1.requires_grad_()
x2 = torch.tensor([2.1,2.2,2.3,2.4])
x2.requires_grad_()

y1 = f1(x1)
y1 = y1.reshape((y1.shape[0] * y1.shape[1]))
I = torch.eye(y1.shape[0])
grad((y1,),(x1,),grad_outputs=(I,), is_grads_batched= True)

(tensor([[1., 3., 5.],
         [2., 4., 6.]]),)

In [18]:
y1.shape

torch.Size([2])

In [15]:
def f3(x):
    return 1*x[0] + 2*x[1] + 3

x3_1 =  torch.tensor([3.1])
x3_2 =  torch.tensor([3.2])
x3_3 =  torch.tensor([3.3])
for x in (x3_1,x3_2,x3_3):
    x.requires_grad_()
x3 = torch.concat([x3_1,x3_2,x3_3])
grad(x3_1 + 2 * x3_2 + 3,x3,allow_unused=True)

(None,)

In [38]:
x4 = torch.tensor([[1,2],[3,4]],dtype=torch.float)
x4.requires_grad_()
y4 = x4 @ torch.tensor([[1,2,3],[4,5,6]],dtype=torch.float)

(J,) = grad(y4.flatten(),x4,grad_outputs = (torch.eye(torch.numel(y4)),),is_grads_batched=True)
J

tensor([[[1., 4.],
         [0., 0.]],

        [[2., 5.],
         [0., 0.]],

        [[3., 6.],
         [0., 0.]],

        [[0., 0.],
         [1., 4.]],

        [[0., 0.],
         [2., 5.]],

        [[0., 0.],
         [3., 6.]]])

In [41]:
J.resize_(y4.shape + x4.shape)

tensor([[[[1., 4.],
          [0., 0.]],

         [[2., 5.],
          [0., 0.]],

         [[3., 6.],
          [0., 0.]]],


        [[[0., 0.],
          [1., 4.]],

         [[0., 0.],
          [2., 5.]],

         [[0., 0.],
          [3., 6.]]]])

In [40]:
y4.shape

torch.Size([2, 3])

In [42]:
J.shape

torch.Size([2, 3, 2, 2])

In [46]:
import torch 

def jacobian(y:torch.Tensor,x:torch.Tensor,need_higher_grad = True) -> torch.Tensor:
    """基于 torch.autograd.grad 函数的更清晰明了的 API，功能是计算一个雅可比矩阵。

    Args:
        y (torch.Tensor): 函数输出向量
        x (torch.Tensor): 函数输入向量
        need_higher_grad (bool, optional): 是否需要计算高阶导数，如果确定不需要可以设置为 False 以节约资源. 默认为 True.

    Returns:
        torch.Tensor: 计算好的“雅可比矩阵”。注意！输出的“雅可比矩阵”形状为 y.shape + x.shape。例如：y 是 n 个元素的张量，y.shape = [n]；x 是 m 个元素的张量，x.shape = [m]，则输出的雅可比矩阵形状为 n x m，符合常见的数学定义。
        但是若 y 是 1 x n 的张量，y.shape = [1,n]；x 是 1 x m 的张量，x.shape = [1,m]，则输出的雅可比矩阵形状为1 x n x 1 x m，如果嫌弃多余的维度可以自行使用 Jac.squeeze() 一步到位。
        这样设计是因为考虑到 y 是 n1 x n2 的张量； 是 m1 x m2 的张量（或者形状更复杂的张量）时，输出 n1 x n2 x m1 x m2 （或对应更复杂形状）更有直观含义，方便用户知道哪一个元素对应的是哪一个偏导。
    """
    (Jac,) = torch.autograd.grad(
        outputs = (y.flatten(),),
        inputs = (x,),
        grad_outputs=(torch.eye(torch.numel(y)),), 
        create_graph=need_higher_grad, 
        allow_unused=True, 
        is_grads_batched=True
    )
    if Jac is None:
        Jac = torch.zeros(size = (y.shape + x.shape))
    else:
        Jac.resize_(size = (y.shape + x.shape))
    return Jac

In [47]:
def f1(x:torch.Tensor):
    W = torch.tensor([[1,2,3],[4,5,6]],dtype=torch.float)
    b = torch.tensor([7,8,9],dtype=torch.float)
    return x @ W + b 

x = torch.tensor([0.1,0.2])
x.requires_grad_()
y = f1(x)

jacobian(y,x)

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])

In [55]:
x = torch.tensor([[0.1,0.2]])
x.requires_grad_()
y = f1(x)
J = jacobian(y,x)

In [56]:
J

tensor([[[[1., 4.]],

         [[2., 5.]],

         [[3., 6.]]]])

In [58]:
J.squeeze()

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])

In [54]:
x = torch.tensor(0.1)
x.requires_grad_()
y = 2 * x + 3
jacobian(y,x)

tensor(2.)

In [72]:

def batched_jacobian(batched_y:torch.Tensor,batched_x:torch.Tensor,need_higher_grad = True) -> torch.Tensor:
    """计算一个批次的雅可比矩阵。
        注意输入的 batched_y 与 batched_x 应该满足一一对应的关系，否则即便正常输出，其数学意义也不明。

    Args:
        batched_y (torch.Tensor): N x y_shape
        batched_x (torch.Tensor): N x x_shape
        need_higher_grad (bool, optional):是否需要计算高阶导数. 默认为 True.

    Returns:
        torch.Tensor: 计算好的一个批次的雅可比矩阵张量，形状为  N x y_shape x x_shape
    """
    sumed_y = batched_y.sum(dim = 0) # y_shape
    J = jacobian(sumed_y,batched_x,need_higher_grad) # y_shape x N x x_shape
    
    dims = list(range(J.dim()))
    dims[0],dims[sumed_y.dim()] = dims[sumed_y.dim()],dims[0]
    J = J.permute(dims = dims) # N x y_shape x x_shape
    return J

def f1(x:torch.Tensor):
    W = torch.tensor([[1,2,3],[4,5,6]],dtype=torch.float)
    b = torch.tensor([7,8,9],dtype=torch.float)
    return x @ W + b 

batched_x = torch.rand((10,2))
batched_x.requires_grad_()
batched_y = f1(batched_x)
batched_J = batched_jacobian(batched_y,batched_x)

In [75]:
batched_J

tensor([[[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[1., 4.],
         [2., 5.],
         [3., 6.]]])